In [ ]:
! pip install pandas
! pip install pydicom

import glob, pylab, pandas as pd
import pydicom, numpy as np
from skimage.transform import resize



In [ ]:
# Show files in data
!ls data

# display label format
df = pd.read_csv('data/stage_1_train_labels.csv')


# 
No = 4
patientId = df['patientId'][No]
dcm_file = 'data/stage_1_train_images/%s.dcm' % patientId
dcm_data = pydicom.read_file(dcm_file)
#print(df.iloc[No])


#print(np.unique(df.Target))
#print(df['patientId'])

img_size = [224,224] # New size of xray images. 
new_df = pd.DataFrame({'Target':[],'Image':[]}) # Define new data frame for images and labels.
unq, idx = np.unique(df['patientId'], return_index = True) # Get only unique entrances from the provided data (some patients occur multiple times)

# Reshape images and match to corresponding label in new dataframe
for i in idx[0:1000]:
    new_df.Target[i] = df.Target[i] # Get label
    patientId = df['patientId'][i] # get patient id from the idx 
    dcm_file = 'data/stage_1_train_images/%s.dcm' % patientId # find the image-file corresponding to the patient id
    dcm_data = pydicom.read_file(dcm_file) # Load the image 
    new_df.Image[i] = resize(dcm_data.pixel_array, output_shape=img_size, mode='reflect', anti_aliasing=True) # resize image
    if not i % 100:
        print(i)
        
new_df.to_csv('data/data.csv')        

#Show an example image
im = new_df.Image[30]
pylab.imshow(im, cmap=pylab.cm.gist_gray)
pylab.axis('off')